In [1]:
from pandas import read_csv
import pandas as pd
import numpy as np
from functools import reduce
from langdetect import detect
from textblob import TextBlob as tb
import nltk
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
raw_data = read_csv("dataset.csv", encoding = "ISO-8859-1", sep=';', low_memory=False)

# Select just some of the columns data
nn_data = raw_data.loc[:, ['TextPost','Date']]
INIT_TOTAL = len(nn_data)

Remove all non english tweets

In [ ]:
languages = []

for text in nn_data['TextPost']:
    try:
        language = detect(text or '')
        languages.append(language)
    except:
        languages.append('unknown')
        
nn_data['Language'] = languages
nn_data = nn_data.loc[nn_data['Language'] == 'en']

Add real Index column to dataset

In [ ]:
nn_data.insert(0, 'Index', range(0, len(nn_data)))

Split Date column into date and time

In [ ]:
temp = pd.DatetimeIndex(nn_data['Date'])
del nn_data['Date']
nn_data['Date'] = temp.date
nn_data['Time'] = temp.time

Define transform_raw_text to make all transformations to tweet text

In [ ]:
def transform_raw_text(text):
    text = re.sub(r'\B\#[^\s]+\b\s*', '', text) # Remove all hashtags
    text = re.sub(r'\B\@[^\s]+\b\s*', '', text) # Remove all people identifications
    text = ' '.join(nltk.word_tokenize(text))
    return text

Calculate Polarities

In [ ]:
polarities_sum = []
polarities_avg = []
total_sentences = []
#languages = []
polarities = []

for text in nn_data['TextPost']:
    #language = detect(text)
    #languages.append(language)
    text = transform_raw_text(text)
    blob = tb(text)
    sentences_polarities = list(map((lambda x: x.sentiment.polarity), blob.sentences))
    p_sum = np.sum(sentences_polarities)
    polarities_sum.append(p_sum)
    polarities_avg.append(p_sum/len(sentences_polarities))
    total_sentences.append(len(sentences_polarities))
    polarities.append(''.join(str(x) + '|' for x in sentences_polarities))

#nn_data['Language'] = languages
nn_data['Sentences'] = total_sentences
nn_data['Polarities'] = polarities
nn_data['PolaritySum'] = polarities_sum
nn_data['PolarityAvg'] = polarities_avg

Show main results

In [ ]:
neg_rows = len(nn_data.loc[nn_data['PolarityAvg'] < 0])
positive_rows = len(nn_data.loc[nn_data['PolarityAvg'] > 0])
total_rows = len(nn_data)
neutral_rows = total_rows - neg_rows - positive_rows

print('Initital Total: {0}; Total EN: {1}; Neutral: {2}; Positive: {3}; Negative: {4}'.format(str(INIT_TOTAL),str(total_rows),str(neutral_rows),str(positive_rows),str(neg_rows)))

Export dataset to csv file

In [ ]:
nn_data.to_csv('first_evaluation_results.csv', sep=';', encoding='utf-8')